# Introduction


As a part of the ***Magellan*** ecosystem, in this tutorial we will show how to use `deepmatcher` together with `Magellan`, to perform an end-to-end Entity Matching (EM) task. Specifically, `Magellan` performs EM in a **two-stage** fashion where given two tables,
1. Perform **blocking** on the two tables by removing obvious non-matching tuple pairs to get a candidate set K.
2. Perform **matching** to predict each pair in K as match or non-match. This stage consists of the following substeps:
    1. *Magellan* first helps the user take a sample S from K.
    2. The user labels all pairs in S as the training data.
    3. A classifer L will be learned on S.
    4. The user applies L to K to predict each pair as match or non-match.

Given the above workflow, `deepmatcher` fits in the step C, which is to take advantage of deep learning to learn a classifer. For the rest of the tutorial, we will use a real example, which is to match songs across two tables from iTunes and Amazon Music, to go through the workflow.

For more information on `Magellan`, please go to the project website: https://sites.google.com/site/anhaidgroup/projects/magellan.

# Step 0. Preparation
In order to use ***Magellan***, we need to first install it. The easiest way is to use "pip" as follow (please consult the package website for other installation options).

In [ ]:
# Install Magellan.
!pip install py_entitymatching

In [1]:
# Once the installation is done, we import Magellan and deepmatcher.
import py_entitymatching as em
import deepmatcher as dm

import os

# Step 1. Load data
We first load the two input tables in the csv format using `Magellan`, that contain songs from iTunes and Amazon Music. These two tables are in the "example" directory included in our package.

In [ ]:
# The path to the two input tables.
path_A = os.path.join('.', 'sample_data', 'itunes-amazon', 'tableA.csv')
path_B = os.path.join('.', 'sample_data', 'itunes-amazon', 'tableB.csv')

In [ ]:
# Load the two tables.
A = em.read_csv_metadata(path_A, key='id')
B = em.read_csv_metadata(path_B, key='id')

In [ ]:
# Basic information about the tables.
print('Number of tuples in A: ' + str(len(A)))
print('Number of tuples in B: ' + str(len(B)))
print('Number of tuples in A X B (i.e the cartesian product): ' + str(len(A)*len(B)))

In [ ]:
# The first few tuples in table A.
A.head()

In [ ]:
# The first few tuples in table B.
B.head()

# Step 2. Block tables to get the candidate set
We first perform blocking on A and B to get a candidate set K, by removing obvious non-matching tuple pairs. `Magellan` supports four different types of blocker: (1) attribute equivalence, (2) overlap, (3) rule-based, and (4) black-box. Typically, users need to mix and match these blockers with the debugging functionality provided in `Magellan` to get a good candidate set. (Developing a good blocker is not the focus of this tutorial. For more information on developing and debugging a blocker, please consult the user manual of `Magellan`.)

Here we show an example of blocking. Observe that matching tuple pairs should have some common words in the album name, so we first create an overlap blocker on the attribute name "Album_Name" with threshold 2, to remove all pairs with word overlap less 2 in that attribute.

In [ ]:
# Create an overlap blocker in Magellan and apply it to A and B to get the candidate set K1 which is in the format of 
# a dataframe. The "l_out_attrs" and "r_out_attrs" parameters indicate the columns that will be included in K1 from A
# and B respectively.
ob = em.OverlapBlocker()
K1 = ob.block_tables(A, B, 'Album_Name', 'Album_Name',
                    l_output_attrs=['Song_Name', 'Artist_Name', 'Album_Name', 'Genre', 'Price', 'CopyRight', 'Time', 'Released'], 
                    r_output_attrs=['Song_Name', 'Artist_Name', 'Album_Name', 'Genre', 'Price', 'CopyRight', 'Time', 'Released'],
                    overlap_size=2)

In [ ]:
# The number of tuple pairs in K1.
len(K1)

In [ ]:
# The first few tuple pairs in K1
K1.head()

Notice that K1 has more than 3 million pair in it, which is too large to considered for matching. So we create an overlap blocking with threshold 1 on the attribute "Artist_Name" for K1, to filter all pairs in K1 that don't share any word in "Artist_Name". And we get the candidate set K2.

In [ ]:
# Create a new overlap blocker to remove pairs from K1 that have no common word in "Artist_Name".
K2 = ob.block_candset(K1, 'Artist_Name', 'Artist_Name', overlap_size=1)

In [ ]:
# The number of tuple pairs in K2.
len(K2)

After the second blocking step, now we have a candidate set K2 with about 170K pairs. But we think it is still a bit larger to consider for matching. So we apply a third blocker, which is an overlap blocker on the attribute "Song_Name", to further reduce the size of the candidate set.

In [ ]:
# Apply the third overlap blocker.
K3 = ob.block_candset(K2, 'Song_Name', 'Song_Name', overlap_size=1)

In [ ]:
# The number of tuples pairs in K3.
len(K3)

Now we have a candidate set with 38K pairs which is reasonable, so we take K3 as the final candidate set. We save the candidate to the disk in the csv format for future reuse.

In [ ]:
path_K = os.path.join('.', 'sample_data', 'itunes-amazon', 'candidate.csv')
K3.to_csv(path_K, index=False)

# Step 3. Match tuple pairs in the candidate set
In this stage we will match tuple pairs in the candidate set to predict each of them as match or non-match. This is the part that `deepmatcher` will be involved. Specifically, it consists of the following steps:
1. Take a sample S from the candidate set K, and label all pairs in S.
2. Train a classifier L using S. Specifically, we will train a classifier using `deepmatcher`.
3. Apply L to the candidiate set K.

## Sample and label the candidate set
We first take a random sample S from the candidate set K using `Magellan`. Here for example, we sample 500 pairs for K. Then we label each pair as match (enter 1) or non-match (enter 0) and use S as the training data.

In [ ]:
# Take a sample of 500 pairs from the candidate set.
S = em.sample_table(K3, 500)

In [ ]:
# Label the sample S in a GUI. Enter 1 for match and 0 for non-match.
G = em.label_table(S, 'gold')

For the purposes of this tutorial, we will load in a pre-labeled dataset (of 539 tuple pairs) included in this package.

In [ ]:
# The path to the labeled data file.
path_G = os.path.join('.', 'sample_data', 'itunes-amazon', 'gold.csv')

In [ ]:
# Load the labeled data into a dataframe.
G = em.read_csv_metadata(path_G, 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')
print('Number of labeled pairs:', len(G))

## Train a classifier using labeled data
Once we have the labeled data, we use `deepmatcher` to train a classifier. The first thing we need to do is to split the data for training purpose. In this example, we split the labeled data into three parts: training, validation and test data, with the ratio of 3:1:1. (For now we only support spliting the labeled data into three parts train/valid/test, where the validation set is used for selecting the best model during the training epochs.) For the purpose of caching data and progressive training, we will first save the split parts to disk in the format of csv files, then load them back in. The cache file will be saved during the loading procedure. For subsequent training runs, the cache file will be used to save preprocessing time on the raw csv files, unless the csv files are modified (in this case, new cache file will be generated).

In [ ]:
# The directory where the data splits will be saved.
split_path = os.path.join('.', 'sample_data', 'itunes-amazon')

In [2]:
# Split labeled data into train, valid, and test csv files to disk, with the split ratio of 3:1:1.
dm.data.split(G, split_path, 'itunes_amz_train.csv', 'itunes_amz_valid.csv', 'itunes_amz_test.csv',
              [3, 1, 1])

NameError: name 'G' is not defined

In [3]:
# Load the training data files from the disk. Ignore the "left_id" and "right_id" 
# columns for data preprocessing.
train, validation, test = dm.process(
    path=os.path.join('.', 'sample_data', 'itunes-amazon'),
    cache='train_cache.pth',
    train='itunes_amz_train.csv',
    validation='itunes_amz_valid.csv',
    test='itunes_amz_test.csv',
    left_prefix='ltable_',
    right_prefix='rtable_',
    id_attr='_id',
    ignore_columns=('ltable_id', 'rtable_id'))

After we get the training data, we can use `deepmatcher` to train a classifier. Here we train a hybrid model.

In [4]:
# Create a hybrid model.
model = dm.MatchingModel(attr_summarizer='sif')

In [5]:
# Train the hybrid model with 10 training epochs, batch size of 16, positive weight 
# of 1.5 (meaning the postive:negtive ratio is 3). We save the best model (with the 
# highest F1 score on the validation set) to 'hybrid_model.pth'.
model.run_train(
    train,
    validation,
    epochs=10,
    batch_size=16,
    best_save_path='sif_model.pth',
    pos_weight=1.5)

* Number of trainable parameters: 662602
===>  TRAIN Epoch 1 :


0% [████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 1 || Run Time:    0.4 | Load Time:    0.5 || F1:  43.14 | Prec:  43.42 | Rec:  42.86 || Ex/s: 373.58

===>  EVAL Epoch 1 :


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 1 || Run Time:    0.1 | Load Time:    0.2 || F1:  22.22 | Prec: 100.00 | Rec:  12.50 || Ex/s: 426.14

* Best F1: 22.22222222222222
Saving best model...
===>  TRAIN Epoch 2 :


...

Finished Epoch 9 || Run Time:    0.1 | Load Time:    0.2 || F1:  86.79 | Prec:  79.31 | Rec:  95.83 || Ex/s: 426.78

===>  TRAIN Epoch 10 :


0% [████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 10 || Run Time:    0.3 | Load Time:    0.5 || F1:  97.44 | Prec:  96.20 | Rec:  98.70 || Ex/s: 375.69

===>  EVAL Epoch 10 :


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 10 || Run Time:    0.1 | Load Time:    0.2 || F1:  84.62 | Prec:  78.57 | Rec:  91.67 || Ex/s: 425.81

Loading best model...


Once we have the trained classifier, we can evaluate the accuracy using the test data.

In [6]:
# Evaluate the accuracy on the test data.
model.run_eval(test)

===>  EVAL Epoch 5 :
Finished Epoch 5 || Run Time:    0.0 | Load Time:    0.2 || F1:  90.00 | Prec:  93.10 | Rec:  87.10 || Ex/s: 436.52



90.0

## Apply the trained classifier to the candidate set

In [20]:
candidate = dm.process(
    path=os.path.join('.', 'sample_data', 'itunes-amazon'),
    unlabeled='cand_sample.csv',
    id_attr='_id',
    left_prefix='ltable_',
    right_prefix='rtable_',
    ignore_columns=('ltable_id', 'rtable_id'))

Load time: 0.09753462765365839
Vocab time: 0.009622064419090748
Metadata time: 0.0002666749060153961


In [21]:
predictions = model.run_prediction(candidate)

RuntimeError: cuda runtime error (59) : device-side assert triggered at /opt/conda/conda-bld/pytorch_1518244507981/work/torch/lib/THC/generic/THCTensorCopy.c:20